## **EXTRAER LOS TICKERS DE LAS EMPRESAS DEL INDICE S&P500**


Este es el primer paso para poder importar los datos financieros de balance, cashflow y PyG.

Es un procedimiento relativamente simple, leeremos los datos desde la web (Wikipedia en este caso) y crearemos un listado en python, sobre el que iteraremos más adelante. 

Repetiremos este mismo proceso en los siguientes cuadernos, por lo que tendremos la oportunidad de revisarlo en caso de que queden algunas dudas. 



In [ ]:
# Preparamos la celda para importar las librerias necesarias para el proyecto, en este caso pandas, numpy y yfinance. Si no se encuentran instaladas, se mostrará un mensaje de error indicando cuál es la librería faltante y cómo instalarla.

# Importamos las librerías necesarias
    # pandas: para manipulación y análisis de datos en forma de tablas (DataFrames)
    # requests: para hacer peticiones HTTP y descargar contenido de páginas web
    # io.StringIO: para manejar cadenas de texto como archivos, útil para leer datos descargados

try:
    import pandas as pd
    import requests
    from io import StringIO
    print("Todas las librerías se han importado correctamente")
except ImportError as e:
    print(f"Error al importar una o más librerías: {e}, es problable que necesites instalar esta libreria usando pip install")

Todas las librerías se han importado correctamente


## Creamos el Dataframe

In [34]:

########---CARGAMOS LOS TICKERS DEL S&P 500 DESDE WIKIPEDIA---########

# URL de la página de Wikipedia que contiene la lista de empresas del S&P 500
url_sp500 = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Definimos un encabezado HTTP (User-Agent) para simular un navegador real.
# Wikipedia bloquea algunas peticiones que no tienen User-Agent.
# Un User-Agent común es el de Mozilla Firefox o Google Chrome, pero aquí usamos uno genérico.
# Un user-Agent es una cadena que identifica el tipo de cliente que hace la petición HTTP, en este caso, un navegador web. Navegamos con Mozilla Firefox en Windows 10, por lo que usamos un User-Agent que simula esa configuración.
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Hacemos la petición GET a Wikipedia y verificamos que sea exitosa
response = requests.get(url_sp500, headers=headers)
response.raise_for_status()  # Lanza un error si la página no respondió correctamente

# Convertimos el HTML de la respuesta en un buffer de texto en memoria.
# Esto evita problemas de pandas/lxml al leer HTML directamente desde URLs en algunas versiones de Python.
html_io = StringIO(response.text)

# Leemos todas las tablas del HTML con pandas
# pd.read_html devuelve una lista de DataFrames; seleccionamos la primera tabla
# que contiene la lista principal de empresas del S&P 500.
sp500 = pd.read_html(html_io)[0]

# Mostramos el DataFrame resultante
sp500

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


Ahora vamos a modelar la información extraida de wikipedia para renombrar la tabla y quedarnos con las columnas que nos interesan

In [30]:
# Renombramos algunas columnas para que tengan nombres más simples e intuitivos.
    # Symbol  -> ticker (símbolo bursátil)
    # Security -> name (nombre de la empresa)
    # GICS Sector -> sector (sector económico)
sp500 = sp500.rename(
    columns={
        'Symbol': 'ticker',
        'Security': 'name',
        'GICS Sector': 'sector'
    }
)

# Seleccionamos únicamente las columnas que nos interesan, eliminando información innecesaria
sp500_clean = sp500[['ticker', 'name', 'sector']]

# En este punto, sp500_clean contiene:
# - ticker: símbolo bursátil de la empresa
# - name: nombre de la empresa
# - sector: sector económico al que pertenece

# visualizamos nuestra tabla con los datos limpios
sp500_clean

,ticker,name,sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology
...,...,...,...
498,XYL,Xylem Inc.,Industrials
499,YUM,Yum! Brands,Consumer Discretionary
500,ZBRA,Zebra Technologies,Information Technology
501,ZBH,Zimmer Biomet,Health Care


Podéis por vuestra cuenta probar con otros indices adaptando este código




_






¿Podemos descargar estos datos en Excel o CSV?

Si, de hecho repetiremos esto más adelante cuando descarguemos los datos financieros de las empresas del selectivo.

In [ ]:
########---OPCIONAL: DECARGAR LA TABLA A EXCEL---########
# Descargamos el DataFrame limpio en un archivo Excel
# path = C:/Tu/Carpeta/sp500_tickers.xlsx
sp500_clean.to_excel("sp500_tickers.xlsx", index=False)

# Si deseais descargarlo en una ruta especifica, podeis usar una ruta absoluta, por ejemplo:
# sp500_clean.to_excel("C:/ruta/a/tu/carpeta/sp500_tickers.xlsx", index=False)

## Creamos la lista de tickers sobre la cual iteraremos

Una lista en Python es una estructura de datos que permite almacenar múltiples elementos, posiblemente de distintos tipos, en una sola variable, manteniendo un orden y permitiendo su modificación.

Ejemplo
numeros = [1, 2, 3, 4, 5]

In [32]:
#Creamos la lista de tickers a partir de la columna 'ticker' del DataFrame limpio, añadimos unique() para evitar duplicados y tolist() para convertir el resultado en una lista de Python
tickers_list = sp500_clean['ticker'].unique().tolist()

# también podeis hacerlo así:
# tickers_list = list(sp500_clean['ticker'].unique())

# Mostramos la lista de tickers
print(len(tickers_list))
print(tickers_list)

503
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APP', 'APTV', 'ACGL', 'ADM', 'ARES', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'XYZ', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CPT', 'CPB', 'COF', 'CAH', 'CCL', 'CARR', 'CVNA', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CIEN', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'FIX', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRH', 'CRWD', 'CCI', 'CSX', 'C

## **Información importante**

Como hemos explicado previamente, obtenemos los tickers del índice S&P500 usando Wikipedia de la fuente. Es importante señalar que tanto la URL, como el contenido de la web pueden verse modificados en el futuro, por lo que hay que estar antentos a posibles cambios que puedan suceder.

De todas maneras, encontrar la URL fue simple, basta con buscar en Google "empresas del S&P500 Wikipedia" y encontraremos la opción más adecuada.

**Ejemplo extra con el Nasdaq 100:**

In [38]:
########---CARGAMOS LOS TICKERS DEL S&P 500 DESDE WIKIPEDIA---########

# URL de la página de Wikipedia que contiene la lista de empresas del S&P 500
nasdaq100 = "https://en.wikipedia.org/wiki/Nasdaq-100"

# Definimos un encabezado HTTP (User-Agent) para simular un navegador real.
# Wikipedia bloquea algunas peticiones que no tienen User-Agent.
# Un User-Agent común es el de Mozilla Firefox o Google Chrome, pero aquí usamos uno genérico.
# Un user-Agent es una cadena que identifica el tipo de cliente que hace la petición HTTP, en este caso, un navegador web. Navegamos con Mozilla Firefox en Windows 10, por lo que usamos un User-Agent que simula esa configuración.
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Hacemos la petición GET a Wikipedia y verificamos que sea exitosa
response = requests.get(nasdaq100, headers=headers)
response.raise_for_status()  # Lanza un error si la página no respondió correctamente

# Convertimos el HTML de la respuesta en un buffer de texto en memoria.
# Esto evita problemas de pandas/lxml al leer HTML directamente desde URLs en algunas versiones de Python.
html_io = StringIO(response.text)

nasdaq100_tickers = pd.read_html(html_io)[4]

nasdaq100_tickers

,Ticker,Company,ICB Industry[14],ICB Subsector[14]
0,ADBE,Adobe Inc.,Technology,Computer Software
1,AMD,Advanced Micro Devices,Technology,Semiconductors
2,ABNB,Airbnb,Consumer Discretionary,Diversified Commercial Services
3,ALNY,Alnylam Pharmaceuticals,Health Care,Biotechnology
4,GOOGL,Alphabet Inc. (Class A),Technology,Computer Software
...,...,...,...,...
96,WBD,Warner Bros. Discovery,Consumer Discretionary,Miscellaneous Amusement & Recreation Services
97,WDC,Western Digital,Technology,Electronic Components
98,WDAY,"Workday, Inc.",Technology,EDP Services
99,XEL,Xcel Energy,Utilities,Power Generation


**Fin del ejercicio** 